In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from config import GptConfig
from model import GPT

In [7]:
torch.cuda.is_bf16_supported()

True

In [8]:
config = GptConfig()

sample = torch.randint(1, 10, size=(1, 5)).to(config.device)
target = torch.randint(1, 10, size=(1, 5)).to(config.device)

gpt = GPT(config).to(config.device)

out = gpt(sample)

In [ ]:
params = {n: p for n, p in gpt.named_parameters()}
params = {n: p for n, p in params.items() if p.requires_grad}

decay_params = [p for n, p in params.items() if p.dim() >= 2]
nodecay_params = [p for n, p in params.items() if p.dim() < 2]

param_grps = [
    {'params': decay_params, 'weight_decay': 0.1, },
    {'params': nodecay_params, 'weight_decay': 0.0, },
]

num_decay_params = sum(p.numel() for p in decay_params)
num_nodecay_params = sum(p.numel() for p in nodecay_params)

print(f"total num of decay parms :{num_decay_params}")
print(f"total num of no decay parms :{num_nodecay_params}")

fused_available = "fused" in inspect.signature(torch.optim.AdamW).parameters
use_fused = fused_available and 'cuda' in device
print(f'Using fused: {use_fused}')

optim.Ada

total num of decay parms :124318464
total num of no decay parms :152401


In [29]:
import inspect



In [24]:
nodecay_params

[Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.,

In [18]:
params

{'tok_embed.weight': Parameter containing:
 tensor([[-0.0215,  0.0119,  0.0007,  ...,  0.0032, -0.0154, -0.0191],
         [ 0.0089,  0.0238, -0.0254,  ..., -0.0099, -0.0032, -0.0025],
         [-0.0362, -0.0131, -0.0127,  ...,  0.0051, -0.0094, -0.0017],
         ...,
         [ 0.0065,  0.0303, -0.0050,  ...,  0.0264, -0.0052,  0.0002],
         [-0.0033, -0.0045, -0.0045,  ...,  0.0057,  0.0078, -0.0487],
         [-0.0025, -0.0039,  0.0049,  ..., -0.0406, -0.0221,  0.0058]],
        device='cuda:0', requires_grad=True),
 'pos_embed.pos_embed.weight': Parameter containing:
 tensor([[ 0.0055, -0.0256, -0.0548,  ..., -0.0241,  0.0219,  0.0677],
         [ 0.0248, -0.0286, -0.0253,  ..., -0.0025,  0.0041, -0.0152],
         [-0.0198, -0.0172,  0.0168,  ...,  0.0515,  0.0099, -0.0141],
         ...,
         [ 0.0334, -0.0058, -0.0293,  ...,  0.0099,  0.0017, -0.0306],
         [ 0.0071,  0.0027,  0.0076,  ...,  0.0209,  0.0340,  0.0221],
         [-0.0293,  0.0337, -0.0220,  ...,  0.01

In [4]:
out['logits'].dtype

torch.float32

In [3]:
from utils import load_checkpoint
load_checkpoint(model=gpt)

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/ckpt.pt'

In [5]:
print(f"total parameters in the model")
total_params = 0
for params in gpt.parameters():
    total_params+=params.numel()
print(total_params)

total parameters in the model
124470865


In [ ]:
inputs = ['hai iam goint to'] * 5
gpt.generate(text=inputs, temperature=0.7, top_k=10)

["hai iam goint to-day;\nAnd, like a story of my joys,\nAnd bid her best array; and there lies\nAnd in their births: where is that's not\nIt is, but not the duke,\nIt shall be wary",
 "hai iam goint to her:\nShe was the loathed to her husband;\nShe was she born a shrew-day;\nAnd she, she, I'll play the church;\nAnd when she, there she, on me;\nI'll tell her",
 "hai iam goint to-day;\nAnd all the house of Lancaster hath been made\nAs well in the highness' the sun.\n\nNORFOLK:\nI thank God, the noble duke had me\nAnd he my daughter's hand.",
 'hai iam goint to-day.\n\nGLOUCESTER:\nWhy, then you would not call me;\nFor I was for you that I was stay.\n\nKING EDWARD IV:\nBrother, then, if I command,\nWhat',
 'hai iam goint to-day?\n\nHENRY BOLINGBROKE:\nIt is thy goodly time it.\n\nDUCHESS OF YORK:\nMyself I remember them well, nor my uncle;\nAnd yet my son']

In [ ]:
from dataset import get_data_loader

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
loader = get_data_loader(tokenizer)

In [ ]:
for x, y in loader:
    print(x, y)
    break

tensor([[   12,   448,   547,  ...,  1867,   338,   534],
        [   40,  2937,    25,  ...,   339,   318, 10657],
        [   25,   198, 39461,  ...,   307,   354,  2903],
        ...,
        [  561,   407,   423,  ...,  8322,  9399,    25],
        [   12,  1820,    12,  ...,    13,   198,   198],
        [  910,     0,  8805,  ...,   815,   307,  6190]]) tensor([[  448,   547,   286,  ...,   338,   534,  1438],
        [ 2937,    25,   198,  ...,   318, 10657,    26],
        [  198, 39461,    11,  ...,   354,  2903,   606],
        ...,
        [  407,   423,   340,  ...,  9399,    25,   198],
        [ 1820,    12,    67,  ...,   198,   198, 18973],
        [    0,  8805,   321,  ...,   307,  6190,    25]])
